## **bronze-companies-cash-flow-statement**
<p>This notebook retrieves raw cash flow statements for listed companies, including key metrics like operating, investing, and financing cash flows. It stores the data in its original form to enable financial analysis and trend identification at later stages.</p>

#### **Importing necessary libraries**

In [1]:
import yfinance as yf
import pandas as pd
import json
import datetime

StatementMeta(, 2173a26a-7bd6-4c4c-8525-f10ec50c5820, 5, Finished, Available, Finished)

#### **Variables**

In [ ]:
file_name = "comp_cash_flow_data"
layer_name = "bronze"

#### **Step 1: Load the company codes (tickers) from the CSV**

In [2]:
codes = spark.read.csv('Files/master_files/company_code.csv', header=True)
short_codes = [
    row['short_code'].strip()
    for row in codes.select('short_code').distinct().collect()
    if row['short_code']
]

StatementMeta(, 2173a26a-7bd6-4c4c-8525-f10ec50c5820, 6, Finished, Available, Finished)

#### **Step 2: Fetch Cash Flow Statement for each ticker**

In [3]:
output = []

for i in short_codes:
    try:
        tickers = yf.Ticker(i)
        csh_flow = tickers.cashflow

        if not csh_flow.empty:
            csh_flow['Company'] = i

            csh_flow.reset_index(inplace=True)
            csh_flow.rename(columns = {'index' : 'Breakdown'}, inplace=True)
            final_csh_flow = csh_flow.melt(id_vars = ['Breakdown','Company'],var_name='Date', value_name='Value')

            output.append(final_csh_flow)
    except Exception as e:
        print(f"An error occurred for {i}: {e}")


StatementMeta(, 2173a26a-7bd6-4c4c-8525-f10ec50c5820, 7, Finished, Available, Finished)

#### **Step 3: Combine all DataFrames and save for Bronze layer**

In [4]:
if output:
    final_csh_flow_df = pd.concat(output, ignore_index=True) 
    
    spark_df = spark.createDataFrame(final_csh_flow_df)

    spark_df.write.mode("overwrite").partitionBy('Date').parquet(f"Files/{layer_name}/{file_name}")

    print(f"[SUCCESS] Stock cash flow data written to Lakehouse")
else:
    print("[WARNING] No data to write.")

StatementMeta(, 2173a26a-7bd6-4c4c-8525-f10ec50c5820, 8, Finished, Available, Finished)

[SUCCESS] Stock cash flow data written to Lakehouse path: Files/bronze/comp_cash_flow_data
